In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
!pip install -U transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 60.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load and clean training data
train_df = pd.read_csv("train.csv")
train_df = train_df.rename(columns=lambda x: x.strip())  # Strip weird column spaces
train_df.columns = train_df.columns.str.strip()  # remove spaces from all columns
train_df = train_df.dropna(subset=["sentiment"])     # now it works safely


# Encode sentiment labels
le = LabelEncoder()
train_df["label_encoded"] = le.fit_transform(train_df["sentiment"])
print(le.classes_)  # Should print ['negative', 'neutral', 'positive']


['negative' 'neutral' 'positive']


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load and clean training data
train_df = pd.read_csv("train.csv")
train_df = train_df.rename(columns=lambda x: x.strip())
train_df = train_df.dropna(subset=["sentiment"])  # if your column was named differently, fix here

# Encode sentiment labels
le = LabelEncoder()
train_df["label_encoded"] = le.fit_transform(train_df["sentiment"])


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=196)

# Use the cleaned train_df
train_dataset = Dataset.from_pandas(train_df[["text", "label_encoded"]])
train_dataset = train_dataset.rename_column("label_encoded", "labels")

# ✅ This creates tokenized_dataset
tokenized_dataset = train_dataset.map(tokenize, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/7632 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

from transformers import TrainingArguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    num_train_epochs=6,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np

# Monkey patch np.array to allow copy=False fallback
def safe_array(*args, **kwargs):
    try:
        return np._original_array(*args, **kwargs)
    except ValueError:
        kwargs.pop("copy", None)
        return np._original_array(*args, **kwargs)

np._original_array = np.array
np.array = safe_array


In [ ]:
trainer.train()


Step,Training Loss
10,1.103700


Step,Training Loss
10,1.103700
20,1.080600


KeyboardInterrupt: 

In [ ]:
# Load test.csv
test_df = pd.read_csv("test.csv").reset_index(drop=True)
test_dataset = Dataset.from_pandas(test_df[["text"]])
tokenized_test = test_dataset.map(tokenize, batched=True)
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask"])


Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

In [ ]:
# Predict
predictions = trainer.predict(tokenized_test)
predicted_class_ids = predictions.predictions.argmax(axis=1)
predicted_labels = le.inverse_transform(predicted_class_ids)

# Build submission
submission = pd.DataFrame({
    "id": test_df["id"],
    "label": predicted_labels
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved!")
submission.head()


✅ submission.csv saved!


,id,label
0,0,positive
1,1,neutral
2,2,neutral
3,3,neutral
4,4,neutral
